In [3]:
# ==========================================
# CHESS OUTCOME PREDICTION — PREPROCESSING
# ==========================================

# --- Imports (everything used in this notebook lives here) ---
from pathlib import Path
import os
import json
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)

# --- Paths ---
NB_DIR = Path.cwd()
CANDIDATES = [
    NB_DIR / "../data/raw_chess_games.csv",
    NB_DIR / "../../data/raw_chess_games.csv",
    NB_DIR / "data/raw_chess_games.csv",
]

RAW_CSV = next((p.resolve() for p in CANDIDATES if p.exists()), None)
if RAW_CSV is None:
    raise FileNotFoundError(
        "Could not find raw_chess_games.csv. Expected it under ../data/ relative to this notebook."
    )

RESULTS_DIR = (NB_DIR / "../results").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# --- Load ---
df = pd.read_csv(RAW_CSV)
print(f"Loaded: {RAW_CSV}")
print("Initial shape:", df.shape)
df.head()

Loaded: E:\Github Projects\chess-outcome-prediction\data\raw_chess_games.csv
Initial shape: (214980, 23)


,game_id,white_username,black_username,white_rating,black_rating,white_country,black_country,result,termination,time_control,time_class,initial_time,increment,date,eco,opening,num_moves,rated,event,source,avg_rating,rating_diff,time_category
0,f57a1313-0f25-11f0-8731-6cfe544c0428,nihalsarin,Hikaru,3222,3307,nihalsarin,hikaru,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Modern-Defense-...,NaN,312.0,True,NaN,chess.com,3264.5,-85.0,blitz
1,d2b8e5d3-0f23-11f0-8731-6cfe544c0428,Hikaru,nihalsarin,3330,3199,hikaru,nihalsarin,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Nimzowitsch-Lar...,NaN,320.0,True,NaN,chess.com,3264.5,131.0,blitz
2,70b20004-0f27-11f0-8731-6cfe544c0428,nihalsarin,Hikaru,3226,3303,nihalsarin,hikaru,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Torre-Attack-Fi...,NaN,334.0,True,NaN,chess.com,3264.5,-77.0,blitz
3,35b89065-0f25-11f0-8731-6cfe544c0428,Hikaru,nihalsarin,3317,3212,hikaru,nihalsarin,1/2-1/2,repetition,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Bishops-Opening...,NaN,274.0,True,NaN,chess.com,3264.5,105.0,blitz
4,23a8b502-0f29-11f0-8731-6cfe544c0428,nihalsarin,Hikaru,3214,3315,nihalsarin,hikaru,1/2-1/2,resignation,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Torre-Attack......,NaN,496.0,True,NaN,chess.com,3264.5,-101.0,blitz


In [4]:
# ==========================================
# 2. Inspect Dataset Info
# ==========================================

# If this cell is run before Cell 1, reload the dataset for safety
if "df" not in globals():
    df = pd.read_csv(RAW_CSV)
    print(f"(Re)loaded: {RAW_CSV}")

print("Shape:", df.shape)
df.info()
display(df.head(10))
display(df.describe(include="all").transpose().head(50))

Shape: (214980, 23)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214980 entries, 0 to 214979
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   game_id         214980 non-null  object 
 1   white_username  214980 non-null  object 
 2   black_username  214980 non-null  object 
 3   white_rating    214980 non-null  int64  
 4   black_rating    214980 non-null  int64  
 5   white_country   200047 non-null  object 
 6   black_country   200047 non-null  object 
 7   result          214980 non-null  object 
 8   termination     200047 non-null  object 
 9   time_control    200047 non-null  object 
 10  time_class      200047 non-null  object 
 11  initial_time    200047 non-null  float64
 12  increment       200047 non-null  float64
 13  date            200047 non-null  object 
 14  eco             197327 non-null  object 
 15  opening         0 non-null       float64
 16  num_moves       200047 non-null  flo

,game_id,white_username,black_username,white_rating,black_rating,white_country,black_country,result,termination,time_control,time_class,initial_time,increment,date,eco,opening,num_moves,rated,event,source,avg_rating,rating_diff,time_category
0,f57a1313-0f25-11f0-8731-6cfe544c0428,nihalsarin,Hikaru,3222,3307,nihalsarin,hikaru,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Modern-Defense-...,NaN,312.0,True,NaN,chess.com,3264.5,-85.0,blitz
1,d2b8e5d3-0f23-11f0-8731-6cfe544c0428,Hikaru,nihalsarin,3330,3199,hikaru,nihalsarin,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Nimzowitsch-Lar...,NaN,320.0,True,NaN,chess.com,3264.5,131.0,blitz
2,70b20004-0f27-11f0-8731-6cfe544c0428,nihalsarin,Hikaru,3226,3303,nihalsarin,hikaru,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Torre-Attack-Fi...,NaN,334.0,True,NaN,chess.com,3264.5,-77.0,blitz
3,35b89065-0f25-11f0-8731-6cfe544c0428,Hikaru,nihalsarin,3317,3212,hikaru,nihalsarin,1/2-1/2,repetition,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Bishops-Opening...,NaN,274.0,True,NaN,chess.com,3264.5,105.0,blitz
4,23a8b502-0f29-11f0-8731-6cfe544c0428,nihalsarin,Hikaru,3214,3315,nihalsarin,hikaru,1/2-1/2,resignation,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Torre-Attack......,NaN,496.0,True,NaN,chess.com,3264.5,-101.0,blitz
5,d1939b12-0f2a-11f0-8731-6cfe544c0428,nihalsarin,Hikaru,3211,3318,nihalsarin,hikaru,1/2-1/2,unknown,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Caro-Kann-Defen...,NaN,572.0,True,NaN,chess.com,3264.5,-107.0,blitz
6,bb63d0f6-0f26-11f0-8731-6cfe544c0428,Hikaru,nihalsarin,3313,3216,hikaru,nihalsarin,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Nimzowitsch-Lar...,NaN,358.0,True,NaN,chess.com,3264.5,97.0,blitz
7,050015c6-0f2a-11f0-8731-6cfe544c0428,Hikaru,nihalsarin,3321,3208,hikaru,nihalsarin,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Trompowsky-Atta...,NaN,486.0,True,NaN,chess.com,3264.5,113.0,blitz
8,38496dee-0f28-11f0-8731-6cfe544c0428,Hikaru,nihalsarin,3309,3220,hikaru,nihalsarin,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Reti-Opening-Ni...,NaN,776.0,True,NaN,chess.com,3264.5,89.0,blitz
9,97019d9d-0f24-11f0-8731-6cfe544c0428,nihalsarin,Hikaru,3210,3319,nihalsarin,hikaru,1-0,checkmate,180,blitz,3.0,0.0,2025-04-01,https://www.chess.com/openings/Indian-Game-Sla...,NaN,264.0,True,NaN,chess.com,3264.5,-109.0,blitz


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
game_id,214980,214980,f57a1313-0f25-11f0-8731-6cfe544c0428,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
white_username,214980,41650,francyIM,263,NaN,NaN,NaN,NaN,NaN,NaN,NaN
black_username,214980,41611,francyIM,269,NaN,NaN,NaN,NaN,NaN,NaN,NaN
white_rating,214980.0,NaN,NaN,NaN,2409.71556,335.325708,348.0,2209.0,2445.0,2637.0,3517.0
black_rating,214980.0,NaN,NaN,NaN,2408.870955,335.117792,112.0,2209.0,2444.0,2636.0,3400.0
white_country,200047,40686,dpopadic,248,NaN,NaN,NaN,NaN,NaN,NaN,NaN
black_country,200047,40612,francyim,252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
result,214980,15,1-0,96830,NaN,NaN,NaN,NaN,NaN,NaN,NaN
termination,200047,8,checkmate,112899,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time_control,200047,89,180,86198,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# ==========================================
# 3. Define Target + Drop IDs/Leakage Columns
# ==========================================

# Map results to target labels (adjust to match your CSV’s result column)
if "result" in df.columns:
    result_map = {"1-0": "white", "0-1": "black", "1/2-1/2": "draw"}
    df["target"] = df["result"].map(result_map)

drop_cols = [
    "game_id", "white_username", "black_username",
    "white_country", "black_country",
    "num_moves", "termination", "opening", "eco"
]

df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

print("After dropping ID/leakage cols:", df.shape)
df.head(3)

After dropping ID/leakage cols: (214980, 15)


,white_rating,black_rating,result,time_control,time_class,initial_time,increment,date,rated,event,source,avg_rating,rating_diff,time_category,target
0,3222,3307,1-0,180,blitz,3.0,0.0,2025-04-01,True,NaN,chess.com,3264.5,-85.0,blitz,white
1,3330,3199,1-0,180,blitz,3.0,0.0,2025-04-01,True,NaN,chess.com,3264.5,131.0,blitz,white
2,3226,3303,1-0,180,blitz,3.0,0.0,2025-04-01,True,NaN,chess.com,3264.5,-77.0,blitz,white


In [6]:
# ==========================================
# 4. Add Calendar Features + Ratings
# ==========================================

# Convert date → calendar fields
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["dayofweek"] = df["date"].dt.dayofweek
    df = df.drop(columns=["date"])

# Rating-based features
if {"white_rating", "black_rating"} <= set(df.columns):
    df["rating_diff"] = df["white_rating"] - df["black_rating"]
    df["avg_rating"] = (df["white_rating"] + df["black_rating"]) / 2

# Convert booleans to integers
if "rated" in df.columns and df["rated"].dtype == bool:
    df["rated"] = df["rated"].astype(int)

print("Columns now:", list(df.columns))
df.head(3)

Columns now: ['white_rating', 'black_rating', 'result', 'time_control', 'time_class', 'initial_time', 'increment', 'rated', 'event', 'source', 'avg_rating', 'rating_diff', 'time_category', 'target', 'year', 'month', 'dayofweek']


,white_rating,black_rating,result,time_control,time_class,initial_time,increment,rated,event,source,avg_rating,rating_diff,time_category,target,year,month,dayofweek
0,3222,3307,1-0,180,blitz,3.0,0.0,True,NaN,chess.com,3264.5,-85,blitz,white,2025.0,4.0,1.0
1,3330,3199,1-0,180,blitz,3.0,0.0,True,NaN,chess.com,3264.5,131,blitz,white,2025.0,4.0,1.0
2,3226,3303,1-0,180,blitz,3.0,0.0,True,NaN,chess.com,3264.5,-77,blitz,white,2025.0,4.0,1.0


In [15]:
# ==========================================
# 5. Handle Missing Values + Compact Encoding (no zip)
# ==========================================

target_col = "target"
feature_cols = [c for c in df.columns if c not in {target_col, "result"}]

cat_cols = [c for c in feature_cols if df[c].dtype == "object"]
num_cols = [c for c in feature_cols if c not in cat_cols]

# Impute numerics
for c in num_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].median())

# Impute categoricals
for c in cat_cols:
    df[c] = df[c].fillna("Unknown")

# Split categoricals:
# - binary → one-hot (adds 1 column max)
# - everything else → integer code (1 column total)
bin_cols   = [c for c in cat_cols if df[c].nunique(dropna=False) == 2]
multi_cols = [c for c in cat_cols if df[c].nunique(dropna=False) >  2]

# One-hot for binary only
df_bin = pd.get_dummies(df[bin_cols], drop_first=True) if bin_cols else pd.DataFrame(index=df.index)

# Integer-code the multi-class categoricals
mappings = {}
for c in multi_cols:
    codes, uniques = pd.factorize(df[c], sort=True)
    # choose minimal int dtype based on #unique
    nuniq = len(uniques)
    if nuniq <= 127:
        dtype = "int8"
    elif nuniq <= 32767:
        dtype = "int16"
    else:
        dtype = "int32"
    df[f"{c}__code"] = codes.astype(dtype)
    mappings[c] = [str(u) for u in uniques.tolist()]

# Downcast numerics to shrink size
for c in num_cols:
    if pd.api.types.is_integer_dtype(df[c]):
        df[c] = pd.to_numeric(df[c], downcast="integer")
    elif pd.api.types.is_float_dtype(df[c]):
        df[c] = pd.to_numeric(df[c], downcast="float")

# Build final frame: compact numerics + binary OHE + codes + target
keep_numeric = [c for c in num_cols] + [c for c in df.columns if c.endswith("__code")]
parts = []
if keep_numeric: parts.append(df[keep_numeric])
if not df_bin.empty: parts.append(df_bin)
parts.append(df[[target_col]])

df_final = pd.concat(parts, axis=1)

# Quick report
print("Binary OHE cols:", bin_cols)
print("Multi-class coded cols:", {c: df[c].nunique() for c in multi_cols})
print("Final shape:", df_final.shape)

mem_mb = df_final.memory_usage(deep=True).sum() / (1024**2)
print(f"Estimated in-memory size: {mem_mb:.2f} MB")

print("Target distribution:")
display(df_final[target_col].value_counts(dropna=False))

Binary OHE cols: ['rated']
Multi-class coded cols: {'time_control': 90, 'time_class': 5, 'event': 1786, 'time_category': 5}
Final shape: (214980, 17)
Estimated in-memory size: 18.70 MB
Target distribution:


target
white    96830
draw     60817
black    42400
NaN      14933
Name: count, dtype: int64

In [17]:
# ==========================================
# 6. Leakage Sanity Check
# ==========================================

banlist = {"num_moves", "termination", "opening", "eco"}
present_banned = [c for c in df_final.columns if c in banlist]

if present_banned:
    df_final = df_final.drop(columns=present_banned, errors="ignore")
    print("Removed banned columns:", present_banned)

print("Final feature count (incl. target):", df_final.shape[1])
df_final.head()

Final feature count (incl. target): 17


,white_rating,black_rating,initial_time,increment,avg_rating,rating_diff,year,month,dayofweek,time_control__code,event__code,time_control__code,event__code,time_class__code,time_category__code,rated_Unknown,target
0,3222,3307,3.0,0.0,3264.5,-85,2025.0,4.0,1.0,30,0,30,0,1,1,False,white
1,3330,3199,3.0,0.0,3264.5,131,2025.0,4.0,1.0,30,0,30,0,1,1,False,white
2,3226,3303,3.0,0.0,3264.5,-77,2025.0,4.0,1.0,30,0,30,0,1,1,False,white
3,3317,3212,3.0,0.0,3264.5,105,2025.0,4.0,1.0,30,0,30,0,1,1,False,draw
4,3214,3315,3.0,0.0,3264.5,-101,2025.0,4.0,1.0,30,0,30,0,1,1,False,draw


In [19]:
# ==========================================
# 7. Save Clean Dataset + Metadata (to data/, uncompressed)
# ==========================================

DATA_DIR = (NB_DIR / "../data").resolve()
DATA_DIR.mkdir(parents=True, exist_ok=True)

clean_csv = DATA_DIR / "clean_chess_games.csv"  # plain CSV, no gzip
df_final.to_csv(clean_csv, index=False)

# Show actual size
size_mb = clean_csv.stat().st_size / (1024**2)
print(f"Saved clean CSV to: {clean_csv}")
print(f"File size: {size_mb:.2f} MB (goal: < 100 MB)")

# Metadata for training notebook
meta = {
    "target_col": target_col,
    "n_rows": int(df_final.shape[0]),
    "n_cols": int(df_final.shape[1]),
    "binary_ohe_cols": bin_cols,
    "coded_categoricals": {k: len(v) for k, v in mappings.items()},
    "dtypes_summary": {c: str(dt) for c, dt in df_final.dtypes.items()}
}
with open(DATA_DIR / "clean_metadata.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2)

df_final.head()

Saved clean CSV to: E:\Github Projects\chess-outcome-prediction\data\clean_chess_games.csv
File size: 14.57 MB (goal: < 100 MB)


,white_rating,black_rating,initial_time,increment,avg_rating,rating_diff,year,month,dayofweek,time_control__code,event__code,time_control__code,event__code,time_class__code,time_category__code,rated_Unknown,target
0,3222,3307,3.0,0.0,3264.5,-85,2025.0,4.0,1.0,30,0,30,0,1,1,False,white
1,3330,3199,3.0,0.0,3264.5,131,2025.0,4.0,1.0,30,0,30,0,1,1,False,white
2,3226,3303,3.0,0.0,3264.5,-77,2025.0,4.0,1.0,30,0,30,0,1,1,False,white
3,3317,3212,3.0,0.0,3264.5,105,2025.0,4.0,1.0,30,0,30,0,1,1,False,draw
4,3214,3315,3.0,0.0,3264.5,-101,2025.0,4.0,1.0,30,0,30,0,1,1,False,draw
